# Final Report

### Introduction

In this project, I hope to create a traveller map of Tokyo that would show the key hotels around the palace and the restuarants nearby. 

### Audience

Travellers who intend to visit Tokyo may find the map handy as they can use it to refer to the suitable hotels and restaurants.

### Data

Foursquare data with details on Tokyo. Data are then cleaned to provide only the name, categories, address, lat, lng, postalCode of the desired areas.

### Methodology 

The following actions are taken to create the map for the travellers. 
1. import libraries.
2. get the geogrphaical details of singapore.
3. search for hotels and clean the dataframe.
4. search for restaurants and clean the dataframe. 
5. generate the traveller map.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [17]:
# The code was removed by Watson Studio for sharing.

Your credentails:


In [3]:
address = 'tokyo'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

35.6828387 139.7594549


In [19]:
# get hotels
search_query = 'Hotel'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [5]:
results = requests.get(url).json()

In [6]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,beenHere.count,beenHere.lastCheckinExpiredAt,beenHere.marked,beenHere.unconfirmedCount,categories,hasPerk,hereNow.count,hereNow.groups,hereNow.summary,hierarchy,...,location.postalCode,location.state,name,referralId,stats.checkinsCount,stats.tipCount,stats.usersCount,stats.visitsCount,venueChains,verified
0,0,0,False,0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,0,[],Nobody here,NaN,...,100-0005,東京都,Palace Hotel Tokyo (パレスホテル東京),v-1574001745,0,0,0,0,[],False
1,0,0,False,0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,0,[],Nobody here,[],...,NaN,東京都,Hotel Arcade (ホテルアーケード),v-1574001745,0,0,0,0,[],False
2,0,0,False,0,"[{'id': '56aa371be4b08b9a8d5734cf', 'name': 'B...",False,0,[],Nobody here,[],...,NaN,東京都,Aoi (葵),v-1574001745,0,0,0,0,[],False
3,0,0,False,0,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",False,0,[],Nobody here,[],...,100-0005,東京都,evian Spa,v-1574001745,0,0,0,0,[],False
4,0,0,False,0,"[{'id': '5032872391d4c4b30a586d64', 'name': 'E...",False,0,[],Nobody here,NaN,...,100-0005,東京都,TESLA SUPERCHARGER in Palace Hotel Tokyo (TESL...,v-1574001745,0,0,0,0,[],False


In [7]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]
clean_dataframe.head()


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,丸の内1-1-1,JP,東京,日本,NaN,261,"[丸の内1-1-1, 千代田区, 東京都, 100-0005, 日本]","[{'label': 'display', 'lat': 35.684644, 'lng':...",35.684644,139.761302,100-0005,東京都,4b05879af964a520ac9b22e3
1,Hotel Arcade (ホテルアーケード),Shopping Mall,丸の内1-1-1,JP,東京,日本,パレスホテル B1F,253,"[丸の内1-1-1 (パレスホテル B1F), 千代田区, 東京都, 日本]","[{'label': 'display', 'lat': 35.684539726631, ...",35.684540,139.761320,NaN,東京都,50b58b02e4b0be8d47ad6128
2,Aoi (葵),Ballroom,丸の内1-1-1,JP,東京,日本,パレスホテル東京 2F,261,"[丸の内1-1-1 (パレスホテル東京 2F), 千代田区, 東京都, 日本]","[{'label': 'display', 'lat': 35.6845839139933,...",35.684584,139.761398,NaN,東京都,5191a011498edf764df5dae2
3,evian Spa,Spa,丸の内1-1-1,JP,東京,日本,パレスホテル東京 5F,269,"[丸の内1-1-1 (パレスホテル東京 5F), 千代田区, 東京都, 100-0005, 日本]","[{'label': 'display', 'lat': 35.68487302074563...",35.684873,139.761077,100-0005,東京都,4fc1d405e4b0d6554a905235
4,TESLA SUPERCHARGER in Palace Hotel Tokyo (TESL...,EV Charging Station,丸の内1-1-1,JP,東京,日本,パレスホテル東京地下1階駐車場内,287,"[丸の内1-1-1 (パレスホテル東京地下1階駐車場内), 千代田区, 東京都, 100-0...","[{'label': 'display', 'lat': 35.68485147271067...",35.684851,139.761452,100-0005,東京都,5b0a112d8e886a002ce03e60


In [20]:
# get restaurants
search_query = 'restaurant'
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [9]:
Rresults = requests.get(url).json()

In [10]:
venues = Rresults['response']['venues']
Restaurant_dataframe = json_normalize(venues)

In [11]:
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]
clean_Restaurant_dataframe

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Coca Restaurant x mango tree cafe,Thai Restaurant,有楽町1-11-1,JP,東京,日本,ビックカメラ有楽町店 6F,877,"[有楽町1-11-1 (ビックカメラ有楽町店 6F), 千代田区, 東京都, 100-000...","[{'label': 'display', 'lat': 35.67554834466681...",35.675548,139.763153,100-0006,東京都,4ba75cb6f964a520348e39e3
1,MOTIF RESTAURANT & BAR,Restaurant,丸の内1-11-1,JP,千代田区,日本,フォーシーズンズホテル丸の内 東京 7F,838,"[丸の内1-11-1 (フォーシーズンズホテル丸の内 東京 7F), 千代田区, 東京都, ...","[{'label': 'display', 'lat': 35.678271, 'lng':...",35.678271,139.766827,100-6277,東京都,55482b82498ead95e681b34b
2,The Restaurant by AMAN (ザ･レストラン by アマン),Mediterranean Restaurant,大手町1-5-6,JP,東京,日本,大手町タワー,600,"[大手町1-5-6 (大手町タワー), 千代田区, 東京都, 日本]","[{'label': 'display', 'lat': 35.68524351388159...",35.685244,139.765400,NaN,東京都,550cd716498ebe3e12462db0
3,Kanoya Athlete Restaurant (東京アスリート食堂),Japanese Restaurant,神田錦町3-21,JP,千代田区,日本,10over9 1F,988,"[神田錦町3-21 (10over9 1F), 千代田区, 東京都, 101-0054, 日本]","[{'label': 'display', 'lat': 35.69157983304175...",35.691580,139.761351,101-0054,東京都,5386a5c8498ee4b654b4ce58
4,Restaurant ベラージュ,French Restaurant,大手町2-2-2,JP,東京,日本,アーバンネット大手町ビル 21F LEVEL XXI,772,"[大手町2-2-2 (アーバンネット大手町ビル 21F LEVEL XXI), 千代田区, ...","[{'label': 'display', 'lat': 35.68584926137846...",35.685849,139.767154,100-0004,東京都,4ff514f764a40b36eae9d0d0
5,Restaurant Rosette (レストラン ロゼット),French Restaurant,丸の内2-1-1,JP,千代田区,日本,明治生命館 B1F,474,"[丸の内2-1-1 (明治生命館 B1F), 千代田区, 東京都, 100-0005, 日本]","[{'label': 'display', 'lat': 35.678963511318, ...",35.678964,139.761636,100-0005,東京都,4d4a279a83eda1cdfae2afb1
6,restaurant prunier,French Restaurant,NaN,JP,東京,日本,NaN,591,"[千代田区, 東京都, 100-0005, 日本]","[{'label': 'display', 'lat': 35.677701, 'lng':...",35.677701,139.761121,100-0005,東京都,5ce38d916fd6260039d78b8d
7,RESTAURANT ASAJI,Restaurant,大手町1-3-2,JP,東京,日本,大手町カンファレンスセンター,687,"[大手町1-3-2 (大手町カンファレンスセンター), 千代田区, 東京都, 100-000...","[{'label': 'display', 'lat': 35.68838167937733...",35.688382,139.762806,100-0004,東京都,4b6f7dabf964a520b6f22ce3
8,家全七福酒家 Seventh Son Restaurant,Chinese Restaurant,丸の内2-4-1,JP,千代田区,日本,丸ビル 36F,412,"[丸の内2-4-1 (丸ビル 36F), 千代田区, 東京都, 100-6336, 日本]","[{'label': 'display', 'lat': 35.681094, 'lng':...",35.681094,139.763484,100-6336,東京都,4baad975f964a520478a3ae3
9,RESTAURANT TOYO,French Restaurant,有楽町1-1-2,JP,東京,日本,東京ミッドタウン日比谷 3F,969,"[有楽町1-1-2 (東京ミッドタウン日比谷 3F), 千代田区, 東京都, 100-000...","[{'label': 'display', 'lat': 35.67412955491569...",35.674130,139.759275,100-0006,東京都,5ab7c00ea6ec986942270b12


In [12]:
hotel_neighbourhood_df = pd.concat([clean_dataframe, clean_Restaurant_dataframe], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Palace Hotel Tokyo (パレスホテル東京),Hotel,丸の内1-1-1,JP,東京,日本,NaN,261,"[丸の内1-1-1, 千代田区, 東京都, 100-0005, 日本]","[{'label': 'display', 'lat': 35.684644, 'lng':...",35.684644,139.761302,100-0005,東京都,4b05879af964a520ac9b22e3
1,Hotel Arcade (ホテルアーケード),Shopping Mall,丸の内1-1-1,JP,東京,日本,パレスホテル B1F,253,"[丸の内1-1-1 (パレスホテル B1F), 千代田区, 東京都, 日本]","[{'label': 'display', 'lat': 35.684539726631, ...",35.684540,139.761320,NaN,東京都,50b58b02e4b0be8d47ad6128
2,Aoi (葵),Ballroom,丸の内1-1-1,JP,東京,日本,パレスホテル東京 2F,261,"[丸の内1-1-1 (パレスホテル東京 2F), 千代田区, 東京都, 日本]","[{'label': 'display', 'lat': 35.6845839139933,...",35.684584,139.761398,NaN,東京都,5191a011498edf764df5dae2
3,evian Spa,Spa,丸の内1-1-1,JP,東京,日本,パレスホテル東京 5F,269,"[丸の内1-1-1 (パレスホテル東京 5F), 千代田区, 東京都, 100-0005, 日本]","[{'label': 'display', 'lat': 35.68487302074563...",35.684873,139.761077,100-0005,東京都,4fc1d405e4b0d6554a905235
4,TESLA SUPERCHARGER in Palace Hotel Tokyo (TESL...,EV Charging Station,丸の内1-1-1,JP,東京,日本,パレスホテル東京地下1階駐車場内,287,"[丸の内1-1-1 (パレスホテル東京地下1階駐車場内), 千代田区, 東京都, 100-0...","[{'label': 'display', 'lat': 35.68485147271067...",35.684851,139.761452,100-0005,東京都,5b0a112d8e886a002ce03e60
5,Club Lounge (パレスホテル クラブラウンジ),Lounge,丸の内1-1-1,JP,東京,日本,パレスホテル,274,"[丸の内1-1-1 (パレスホテル), 千代田区, 東京都, 100-0005, 日本]","[{'label': 'display', 'lat': 35.684640793411, ...",35.684641,139.761526,100-0005,東京都,5688c05f498e2d8462077793
6,SWEETS & DELI,Dessert Shop,丸の内1-1-1,JP,東京,日本,パレスホテル東京 ホテルアーケード,262,"[丸の内1-1-1 (パレスホテル東京 ホテルアーケード), 千代田区, 東京都, 日本]","[{'label': 'display', 'lat': 35.68441849929843...",35.684418,139.761612,NaN,東京都,4fb4ec036d8660d8e09a490c
7,Ascott Marunouchi Gym,Gym / Fitness Center,1-1-1 Otemachi,JP,東京,日本,NaN,526,"[1-1-1 Otemachi, 千代田区, 東京都, 100-0004, 日本]","[{'label': 'display', 'lat': 35.686707, 'lng':...",35.686707,139.762800,100-0004,東京都,5c879cf3135b39002cf8aaf6
8,Coca Restaurant x mango tree cafe,Thai Restaurant,有楽町1-11-1,JP,東京,日本,ビックカメラ有楽町店 6F,877,"[有楽町1-11-1 (ビックカメラ有楽町店 6F), 千代田区, 東京都, 100-000...","[{'label': 'display', 'lat': 35.67554834466681...",35.675548,139.763153,100-0006,東京都,4ba75cb6f964a520348e39e3
9,MOTIF RESTAURANT & BAR,Restaurant,丸の内1-11-1,JP,千代田区,日本,フォーシーズンズホテル丸の内 東京 7F,838,"[丸の内1-11-1 (フォーシーズンズホテル丸の内 東京 7F), 千代田区, 東京都, ...","[{'label': 'display', 'lat': 35.678271, 'lng':...",35.678271,139.766827,100-6277,東京都,55482b82498ead95e681b34b


### Results

In [13]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

### Discussion

The codes successfully show the key hotels around the palace in Tokyo and the restuarants nearby. 

I wanted to include the following details in the project but was unable to scarpe the data from yelp or trip advisor:
- the specific cuisine it serves
- self-service or table service (casual or fine dining)
- restaurat atmosphere and dress code
- budget and average spending
- reviews and ratings

The map would be much more comprehensive and informative if I have access to such these datasets.

### Conclusion 

It was my first attempt to create a data-project of this scale. I have some diffculties in obtaining all the data I intended to get. As a result I was unable to build a more comprehensive map.

if there data are available, the traveller map would definitely be more useful to travellers.